We use DeepLabCut to train a single network to track key points of monkey hand from multiple camera views.
\
This notebook is based on a template (Demo_yourowndata.ipynb)from DeepLabCut Github (https://github.com/DeepLabCut/DeepLabCut).

It illustrates how to: 
- prepare videos for DLC (cropping, down sampling if needed)
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video
- plot the trajectories

Note:\
Open the gui in the terminal if this notebook fails to open the gui.

Reference: \
Paper: https://www.nature.com/articles/s41596-019-0176-0 
\
Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf

## Data preparation
- make videos from images
- crop and down sample videos

In [ ]:
#%% Make videos from images if needed.
# DeepLabCut requires videos not images.

import cv2
import os

video_main_dir = r'E:\Monkey_project_videos'
# session_name = ['2023-08-03_15-33-29', '2023-08-03_15-38-37']
session_name = ['Calibration_230804']
camera_name_dic = {"19472072": "cam-A", "19472089": "cam-B"}
video_save_path = r'E:\Monkey_project_videos\Calibration_230804'
for s in session_name:
    for i, (k, v) in enumerate(camera_name_dic.items()):
        image_folder = os.path.join(video_main_dir,s,v)
        # video_name = os.path.join(video_save_path, 'cam' + k + '_video' + s + '.avi')
        video_name = os.path.join(video_save_path, 'cam' + k + '_video' + s + '.mp4')

        images = [img for img in os.listdir(image_folder) if img.endswith(".tiff")]
        frame = cv2.imread(os.path.join(image_folder, images[0]))
        height, width, layers = frame.shape

        video = cv2.VideoWriter(video_name,  #Provide a file to write the video to
            # fourcc=cv2.VideoWriter_fourcc('X','V','I','D'), # code for avi
            fourcc = cv2.VideoWriter_fourcc(*'mp4v'), # code for mp4
            fps=20,                                        #How many frames do you want to display per second in your video?
            frameSize=(width, height))                     #The size of the frames you are writing

        for image in images:
            video.write(cv2.imread(os.path.join(image_folder, image)))

        cv2.destroyAllWindows()
        video.release()


In [ ]:
# Crop and down sample videos
# image_width = 1920 # pixels
# image_height = 1200 # pixels
# video_save_path = r'E:\Monkey_project_videos\2023-08-03-videos'
# videos = [vid for vid in os.listdir(video_save_path) if vid.endswith(".avi")]
# down_sample_ratio = 0.8
# for v in videos:
#     video_name = os.path.join(video_save_path, v)
#     downsampled_video_name = deeplabcut.DownSampleVideo(video_name,
#                                width=image_width*down_sample_ratio,
#                                height=image_height*down_sample_ratio,
#                                outsuffix='_downsampled')
#     cropped_downsampled_video_name = deeplabcut.CropVideo(downsampled_video_name,
#                                                      origin_x=250, origin_y=100,
#                                                      width=800, height=800,
#                                                      outsuffix='_cropped')
# deeplabcut.DownSampleVideo?
# deeplabcut.CropVideo?

In [ ]:
# # Crop 
# image_width = 1920 # pixels
# image_height = 1200 # pixels
# video_save_path = r'E:\Monkey_project_videos\2023-08-03-videos'
# videos = [vid for vid in os.listdir(video_save_path) if vid.endswith(".avi")]
# for v in videos:
#     video_name = os.path.join(video_save_path, v)
#     cropped_video_name = deeplabcut.CropVideo(video_name,
#                                              origin_x=500, origin_y=150,
#                                              width=850, height=850,
#                                              outsuffix='_cropped')
    

## Create a new project

It is always good idea to keep the projects separate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [1]:
import deeplabcut

Loading DLC 2.3.5...


In [2]:
task='Testing2Cam' # Enter the name of your experiment Task
experimenter='Yiting' # Enter the name of the experimenter
video=['E:/Monkey_project_videos/2023-08-03_videos'] # Enter the paths of your videos OR FOLDER you want to grab frames from.

# path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True,working_directory="C:/Users/Yiting/Desktop",) 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
path_config_file = r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

## Now, go edit the config.yaml file that was created! 
Add your body part labels, edit the number of frames to extract per video, etc. 

#### Note that you can see more information about ANY function by adding a ? at the end,  i.e. 

In [ ]:
deeplabcut.extract_frames?

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) ('uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
%matplotlib inline
#there are other ways to grab frames, such as uniformly; please see the paper:

#AUTOMATIC:
deeplabcut.extract_frames(path_config_file) 

In [ ]:
#AND/OR:
#SELECT RARE EVENTS MANUALLY:
%gui wx
deeplabcut.extract_frames(path_config_file,'manual')

In [ ]:
import matplotlib
matplotlib.use('Agg',warn=False, force=True)
from matplotlib import pyplot as plt
matplotlib.get_backend()

## Label the extracted frames

Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
%gui wx
deeplabcut.label_frames(path_config_file)

## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [ ]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)
#remember, there are several networks you can pick, the default is resnet-50!

## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file)

In [ ]:
deeplabcut.train_network?

## Start evaluating
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
videofile_path = [r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos'] #Enter a folder OR a list of videos to analyze.
# videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

In [ ]:
deeplabcut.analyze_videos?

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [ ]:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,[r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-43-34.avi'],
                                 p_bound = 0.7) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [7]:
deeplabcut.create_labeled_video?

Signature:
deeplabcut.create_labeled_video(
    config,
    videos,
    videotype='',
    shuffle=1,
    trainingsetindex=0,
    filtered=False,
    fastmode=True,
    save_frames=False,
    keypoints_only=False,
    Frames2plot=None,
    displayedbodyparts='all',
    displayedindividuals='all',
    codec='mp4v',
    outputframerate=None,
    destfolder=None,
    draw_skeleton=False,
    trailpoints=0,
    displaycropped=False,
    color_by='bodypart',
    modelprefix='',
    init_weights='',
    track_method='',
    superanimal_name='',
    pcutoff=0.6,
    skeleton=[],
    skeleton_color='white',
    dotsize=8,
    colormap='rainbow',
    alphavalue=0.5,
    overwrite=False,
)
Docstring:
Labels the bodyparts in a video.

Make sure the video is already analyzed by the function
``deeplabcut.analyze_videos``.

Parameters
----------
config : string
    Full path of the config.yaml file.

videos : list[str]
    A list of strings containing the full paths to videos for analysis or a path
 

In [3]:
videofile_path = [r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos'] #Enter a folder OR a list of videos to analyze.

In [4]:
# update pcutoff in config_file
deeplabcut.create_labeled_video(path_config_file,videofile_path,filtered=False,draw_skeleton=True,pcutoff=0.2,overwrite=True)

Analyzing all the videos in the directory...
Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-38-21.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-38-21.avi and data.
Duration of video [s]: 11.3, recorded with 100 fps!
Overall # of frames: 1130 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 1130/1130 [00:20<00:00, 53.83it/s]


Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29.avi and data.
Duration of video [s]: 8.83, recorded with 100 fps!
Overall # of frames: 883 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 883/883 [00:20<00:00, 43.96it/s]


Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-43-34.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-43-34.avi and data.
Duration of video [s]: 11.38, recorded with 100 fps!
Overall # of frames: 1138 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


...

Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-38-37.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-38-37.avi and data.
Duration of video [s]: 8.21, recorded with 100 fps!
Overall # of frames: 821 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 821/821 [00:15<00:00, 53.69it/s]


Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29.avi and data.
Duration of video [s]: 8.83, recorded with 100 fps!
Overall # of frames: 883 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 883/883 [00:16<00:00, 53.50it/s]


[True, True, True, True, True, True, True, True, True, True]

In [7]:
# save_frames = True -> very slow
videofile_path = [r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29.avi',
                 r'C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29.avi']
deeplabcut.create_labeled_video(path_config_file,videofile_path,filtered=False,
                                draw_skeleton=True,pcutoff=0.2,overwrite=True,
                               save_frames=True)

Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29.avi and data.
Duration of video [s]: 8.83, recorded with 100.0 fps!
Overall # of frames: 883 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 883/883 [11:15<00:00,  1.31it/s]


Labeled video C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472072_video2023-08-03_15-33-29DLC_resnet50_Testing2CamAug5shuffle1_100000_labeled.mp4 successfully created.
Starting to process video: C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29.avi
Loading C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29.avi and data.
Duration of video [s]: 8.83, recorded with 100.0 fps!
Overall # of frames: 883 with cropped frame dimensions: 1920 1200
Generating frames and creating video.


100%|██████████| 883/883 [13:22<00:00,  1.10it/s]


Labeled video C:\Users\Yiting\Desktop\Testing2Cam-Yiting-2023-08-05\videos\cam19472089_video2023-08-03_15-33-29DLC_resnet50_Testing2CamAug5shuffle1_100000_labeled.mp4 successfully created.


[True, True]

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)